In [62]:
import sys
import os
# Get the current script's directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)
parent_dir

'd:\\algothon2025\\algothon2025'

In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from ta.momentum import RSIIndicator, StochasticOscillator, ROCIndicator, WilliamsRIndicator
from ta.trend import MACD


In [64]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers

In [65]:
from preprocess import COMMISSION_RATE, extract_features2
BEFORE = 50
AHEAD = 50
TA_WINDOW = 50

# Per stock extraction
def split_seq(price_data: pd.DataFrame, stock: int):
    length = price_data[stock].shape[0]
    price_data.index = list(range(length))
    X = []
    y = []

    rsi_series, rsi_series2, macd_signal, macd_signal2, macd_signal3, \
        stoch_sign, stoch_sign2, roc_vals, roc_vals2, roc_vals3, \
        williamR_vals, williamR_vals2 = extract_features2(price_data[stock])
    # print(rsi_series)
    # print(rsi_series.shape)
    for i in range(TA_WINDOW + BEFORE - 1, length - AHEAD):
        # Try 1 price/pct: Accuracy 0.57%, seems to do 0R
        # Try all price/pct: Overfit
        # X.append(price_data.iloc[(i - BEFORE + 1):(i+1), :].pct_change().dropna().to_numpy().reshape((BEFORE - 1, -1)))
        
        # Try feature extractions for single price
        current = []
        for j in range(BEFORE):
            current.append([rsi_series[i-j], rsi_series2[i-j],
                            macd_signal[i-j], macd_signal2[i-j], macd_signal3[i-j],
                            stoch_sign[i-j], stoch_sign2[i-j],
                            roc_vals[i-j], roc_vals2[i-j], roc_vals3[i-j],
                            williamR_vals[i-j], williamR_vals2[i-j]])
        X.append(np.array(current))
        return_pct = (price_data[stock][i+AHEAD] - price_data[stock][i]) / price_data[stock][i]
        if abs(return_pct) < COMMISSION_RATE:
            y.append(0)
        else:
            if return_pct > 0:
                y.append(1)
            else:
                y.append(-1)
    # print(X)
    X_df = np.array(X)
    y_df = np.array(y)
    y_df = tf.one_hot(y_df + 1, depth=3)
    return X_df, y_df


In [66]:
df = pd.read_csv("../prices.txt", sep="\\s+", header=None, index_col=None)
df.index = np.arange(df.shape[0])
df.rename(columns=lambda c: int(c), inplace=True)

In [67]:
total = df.shape[0]
train = df.iloc[:int(total * 0.75)]
test = df.iloc[int(total * 0.75):]

In [68]:
STOCK = 5
X_train, y_train = split_seq(train, STOCK)
X_train.shape

(601, 50, 12)

In [69]:
X_train

array([[[ 4.47288561e+01,  4.84445202e+01, -3.08789032e-02, ...,
          6.68543279e-01, -6.34615385e+01, -7.03125000e+01],
        [ 3.24646696e+01,  4.35183948e+01, -3.82793528e-02, ...,
         -7.68156425e-01, -1.00000000e+02, -1.00000000e+02],
        [ 4.22295911e+01,  4.86296091e+01, -2.41614152e-02, ...,
         -2.78551532e-01, -1.00000000e+02, -8.40000000e+01],
        ...,
        [ 4.04492599e+01,  4.82606394e+01, -6.22870215e-02, ...,
         -3.83408853e-01, -1.00000000e+02, -8.62385321e+01],
        [ 4.34619908e+01,  5.01561553e+01, -4.91329371e-02, ...,
         -6.23484586e-01, -9.43181818e+01, -7.61467890e+01],
        [ 4.88167641e+01,  5.34176972e+01, -2.91840410e-02, ...,
         -4.48275862e-01, -7.38636364e+01, -5.28455285e+01]],

       [[ 4.53096503e+01,  4.86924351e+01, -2.37965294e-02, ...,
          3.49528137e-02, -5.65217391e+01, -6.87500000e+01],
        [ 4.47288561e+01,  4.84445202e+01, -3.08789032e-02, ...,
          6.68543279e-01, -6.34615385e

In [70]:
X_train[0]

array([[ 4.47288561e+01,  4.84445202e+01, -3.08789032e-02,
        -1.37058362e-02, -3.06629033e-02,  1.21794872e+01,
         4.99820572e+01, -1.54852030e+00, -3.49406010e-02,
         6.68543279e-01, -6.34615385e+01, -7.03125000e+01],
       [ 3.24646696e+01,  4.35183948e+01, -3.82793528e-02,
        -3.00792440e-02, -2.97231733e-02,  0.00000000e+00,
         5.70133072e+01, -1.62686051e+00, -5.94613501e-01,
        -7.68156425e-01, -1.00000000e+02, -1.00000000e+02],
       [ 4.22295911e+01,  4.86296091e+01, -2.41614152e-02,
        -1.87389068e-02, -1.28623168e-02,  7.84313725e+00,
         6.59021961e+01, -1.10497238e+00,  1.39860140e-01,
        -2.78551532e-01, -1.00000000e+02, -8.40000000e+01],
       [ 4.68405303e+01,  5.06861208e+01, -1.76484332e-02,
        -1.55809130e-02, -3.15771014e-03,  1.66666667e+01,
         7.43021961e+01,  0.00000000e+00,  1.26939351e+00,
        -2.77777778e-01, -1.00000000e+02, -6.80000000e+01],
       [ 5.19451061e+01,  5.28076594e+01, -1.1748792

In [71]:
y_train

<tf.Tensor: shape=(601, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)>

In [72]:
y_train.numpy().sum(axis=0)

array([296.,   3., 302.], dtype=float32)

In [73]:
model = keras.models.Sequential()
model.add(layers.Input(shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.SimpleRNN(units=5, activation="relu", return_sequences=True))
model.add(layers.SimpleRNN(units=5, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dense(3))

In [74]:
model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"]
)

In [75]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 50, 5)             90        
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 5)                 55        
                                                                 
 batch_normalization_3 (Bat  (None, 5)                 20        
 chNormalization)                                                
                                                                 
 dense_3 (Dense)             (None, 3)                 18        
                                                                 
Total params: 183 (732.00 Byte)
Trainable params: 173 (692.00 Byte)
Non-trainable params: 10 (40.00 Byte)
_________________________________________________________________


In [76]:
y_train.shape

TensorShape([601, 3])

In [77]:
X_test, y_test = split_seq(test, STOCK)
X_test.shape

(101, 50, 12)

In [78]:
model.fit(X_train, y_train, batch_size=30, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
21/21 [==============================] - 2s 23ms/step - loss: 1.1550 - accuracy: 0.3943 - val_loss: 27.3629 - val_accuracy: 0.0000e+00
Epoch 2/100
21/21 [==============================] - 0s 12ms/step - loss: 1.0896 - accuracy: 0.4476 - val_loss: 8.1562 - val_accuracy: 0.0000e+00
Epoch 3/100
21/21 [==============================] - 0s 13ms/step - loss: 1.0476 - accuracy: 0.5042 - val_loss: 4.7525 - val_accuracy: 0.0000e+00
Epoch 4/100
21/21 [==============================] - 0s 12ms/step - loss: 1.0139 - accuracy: 0.5308 - val_loss: 4.7836 - val_accuracy: 0.0000e+00
Epoch 5/100
21/21 [==============================] - 0s 12ms/step - loss: 0.9694 - accuracy: 0.5973 - val_loss: 3.6409 - val_accuracy: 0.0000e+00
Epoch 6/100
21/21 [==============================] - 0s 12ms/step - loss: 0.9456 - accuracy: 0.6057 - val_loss: 1.7334 - val_accuracy: 0.2772
Epoch 7/100
21/21 [==============================] - 0s 11ms/step - loss: 0.9153 - accuracy: 0.6506 - val_loss: 1.9385 - val_ac

In [84]:
y_test

<tf.Tensor: shape=(101, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
     

In [85]:
y_pred = model.predict(X_test)

4/4 [==============================] - 0s 3ms/step


In [86]:
y_pred_label = np.argmax(y_pred, axis=1)
y_pred_label

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [87]:
y_test_label = np.argmax(y_test, axis=1)
y_test_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [88]:
print(accuracy_score(y_true=y_test_label, y_pred=y_pred_label))

0.42574257425742573
